###  Filters used to generate final symbols list (959 symbols out of ~2800):

1. Started with **QCSymbols** as it includes following criteria (as of start 2024):
    1. MarketCap > 0  # Excludes negative equity
    2. LastPrice >= 5  # Excludes penny stocks
    3. DollarVolume > 1,000,000  # Liquidity
    4. IsShortable == 'Yes'  # Shortable
    5. Is a stock (since QC News data has ETFs etc)

2. NYSE/NASDAQ Listed (used Polygon)
3. Options available (used yfinance to check)
4. Symbols available on Polygon (used Polygon)
5. News Frequency (> 100 news from 2017 through 2023) (used QCnewsSymbols.csv)
6. sec-api provides sector/industry (modern) & sicSector/sicIndustry (legacy) - using QC mapping for corresponding ETFs
7. using CIK as unique identifier (guaranteed unique by SEC) instead of tickers which can change or be reused

TODO:
 - Out of these 959 symbols, exclude symbols that are not available on:
    - sec-api
    - earningscall.biz
    - IBKR


**Others to consider (not used to filter):**
1. News Impact
2. Volume Spike
3. News Predictability (LLM)

In [1]:
from utils.fetchLocalcsv import load_local_csv 
from utils.misc import get_unique_items
import pandas as pd
import numpy as np

In [2]:
file_path1 = 'QCnewsSymbols.csv'
qc_symbols = load_local_csv(file_path1)
print(f"Unique exchanges: {get_unique_items(qc_symbols, 'Exchange', show_items=True)}")
print(f"Unique symbols: {get_unique_items(qc_symbols, 'SymbolValue', show_items=True)}")

Loaded QCnewsSymbols.csv with 2882 rows and 33 columns
Unique exchanges: (15, ['ASE', 'BATS', 'BRU', 'CSE', 'ETR', 'LIM', 'LON', 'MIL', 'NAS', 'NYS', 'OSL', 'SWX', 'TAE', 'TSE', 'TSX'])
Unique symbols: (2882, ['A', 'AA', 'AACT', 'AAL', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AB', 'ABBV', 'ABCB', 'ABCL', 'ABG', 'ABM', 'ABNB', 'ABR', 'ABT', 'ACA', 'ACAD', 'ACCD', 'ACCO', 'ACDC', 'ACEL', 'ACGL', 'ACHC', 'ACHR', 'ACI', 'ACIC', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACRE', 'ACT', 'ACVA', 'ADBE', 'ADC', 'ADI', 'ADM', 'ADNT', 'ADP', 'ADPT', 'ADSK', 'ADT', 'ADTN', 'ADUS', 'AEE', 'AEHR', 'AEIS', 'AEL', 'AEM', 'AEO', 'AEP', 'AER', 'AES', 'AESI', 'AFCG', 'AFG', 'AFL', 'AFRM', 'AFYA', 'AG', 'AGCO', 'AGI', 'AGIO', 'AGL', 'AGM', 'AGNC', 'AGO', 'AGR', 'AGRO', 'AGS', 'AGTI', 'AGX', 'AGYS', 'AHCO', 'AHH', 'AI', 'AIG', 'AIN', 'AIR', 'AIRC', 'AIRS', 'AIT', 'AIV', 'AIZ', 'AJG', 'AJX', 'AKAM', 'AKR', 'AKRO', 'AL', 'ALB', 'ALC', 'ALCC', 'ALCO', 'ALCY', 'ALE', 'ALEC', 'ALEX', 'ALG', 'AL

#### qc_news: Session returns calculated from Polygon

In [3]:
file_path2 = '../Returns/SessionReturns_QC.csv' # Session returns calculated from Polygon
qc_news = load_local_csv(file_path2, keep_cols=['symbols', 'Unnamed: 0', 'returns', 'volume_return', 'return_SPY', 'originalTime'], na_subset=['returns'])
# Add year column
qc_news = qc_news.assign(year=lambda x: pd.to_datetime(x.originalTime, utc=True).dt.year).drop('originalTime', axis=1)
qc_news.index = qc_news['Unnamed: 0']
qc_news.index.name = 'id'
qc_news.drop(columns=['Unnamed: 0'], inplace=True)
qc_news['Idiosyncratic_Return'] = qc_news['returns'] - qc_news['return_SPY']
qc_news['ReturnDirectionvsS&P'] = np.where((qc_news['returns'] * qc_news['return_SPY']) >= 0, 'Same', 'Opposite')
qc_news.head()

Loaded ../Returns/SessionReturns_QC.csv with 310167 rows and 6 columns


,symbols,volume_return,return_SPY,returns,year,Idiosyncratic_Return,ReturnDirectionvsS&P
id,,,,,,,
2,PFGC,0.160000,0.108205,0.53,2017,0.421795,Same
3,AA,21.690001,0.084141,2.50,2017,2.415859,Same
4,HLI,10.100000,0.084141,-0.22,2017,-0.304141,Opposite
5,STLD,113.180000,0.084141,-1.95,2017,-2.034141,Opposite
6,NUE,17.139999,0.084141,-1.31,2017,-1.394141,Opposite


#### News Frequency by year

In [4]:
print(pd.concat([qc_news.year.value_counts(), qc_news.year.value_counts(normalize=True).mul(100).round(1)], axis=1, keys=['Count', '%']))

      Count     %
year             
2023  93818  30.2
2022  82699  26.7
2021  61954  20.0
2020  42669  13.8
2018  15694   5.1
2019   9418   3.0
2017   3915   1.3


### Filtering 1
1. Started with QC Symbols fetched in start 2024 (MarketCap>0, LastPrice>=5, DollarVolume>1M, IsShortable='Yes')
2. Included only symbols that are present in Polygon (removing rows without polygon returns)
3. Filtered out symbols that are not listed on NYSE/NASDAQ

In [5]:
print(f"Total Potential Universe: {get_unique_items(qc_symbols, 'SymbolValue', show_items=False)}")

# Since returns in qc_news are from Polygon, and we remove rows with no returns above 
# (instead of this - we could even query Polygon database) 
# But this way we also filter out symbols that had no news from 2017 thorough 2023 (and we need 1 year of historical earnings)
filter_symbols = qc_symbols.PrimarySymbol.isin(qc_news.symbols)
qc_symbols = qc_symbols[filter_symbols]
print(f"Universe considering Polygon inclusion: {get_unique_items(qc_symbols, 'PrimarySymbol', show_items=False)}")

filter_exchange = qc_symbols.Exchange.isin(['NYS','NAS'])
qc_symbols = qc_symbols[filter_exchange]
print(f"Universe after filtering exchange: {get_unique_items(qc_symbols, 'PrimarySymbol', show_items=False)}")


Total Potential Universe: 2882
Universe considering Polygon inclusion: 2085
Universe after filtering exchange: 2019


### Filtering 2
1. Remove symbols with no Options (yfinance not 100% reliable but can do for now since removes only few (61) symbols)

In [6]:
# # Takes 5 minutes
# from utils.misc import check_options_yfinance
# options_symbols = check_options_yfinance(qc_symbols.PrimarySymbol.to_list())

# # Save options symbols to file
# with open('options_symbols.txt', 'w') as f:
#     for symbol in options_symbols:
#         f.write(f"{symbol}\n")

In [7]:
options_symbols = pd.read_csv('options_symbols.txt', header=None)[0].values.tolist()
qc_symbols = qc_symbols[qc_symbols.isin(options_symbols)]
print(f"Universe after filtering options: {get_unique_items(qc_symbols, 'PrimarySymbol', show_items=False)}")

Universe after filtering options: 1958


### Filtering 3
1. Sort based on News Frequency & Returns and remove symbols with least Potential

- remove stocks with < 100 news :from 2017 through 2023 (6 - 7 years)




In [8]:
qc_news.head()

,symbols,volume_return,return_SPY,returns,year,Idiosyncratic_Return,ReturnDirectionvsS&P
id,,,,,,,
2,PFGC,0.160000,0.108205,0.53,2017,0.421795,Same
3,AA,21.690001,0.084141,2.50,2017,2.415859,Same
4,HLI,10.100000,0.084141,-0.22,2017,-0.304141,Opposite
5,STLD,113.180000,0.084141,-1.95,2017,-2.034141,Opposite
6,NUE,17.139999,0.084141,-1.31,2017,-1.394141,Opposite


### Calculate Returns Summary - by symbol

In [9]:
# Get counts per symbol
pd.set_option('display.max_rows', None)

# Group by symbols and calculate statistics including year-based counts
summary = (qc_news
          .groupby('symbols')
          .agg({
              'Idiosyncratic_Return': ['count', lambda x: abs(x).mean(), 'max', 'min'],
              'year': lambda x: sum(x.isin([2022, 2023]))  # Count of recent news
          }))

# Flatten column names
summary.columns = ['count', 'avg_abs_return', 'max_return', 'min_return', 'recent_count']

# Filter out symbols with less than 100 news
# summary = summary[summary['count'] > 100]  # leaves 959 symbols

# Filter out symbols with less than 2% return
# summary = summary[summary['avg_abs_return'] > 1] # leaves 340 symbols for > 2 & 770 for > 1

# Filter based on both total count and recent news
summary = summary[
    (summary['count'] > 100) & 
    (summary['recent_count'] > 50)
]

# Calculate weighted score with count-based dampening
summary['weighted_score'] = (
    np.log1p(summary['count']) * 
    np.log1p(summary['avg_abs_return']) * 
    np.log1p(summary['max_return'] - summary['min_return']))  # log1p won't overweight one-off extreme moves


# Sort by weighted score
summary.sort_values('weighted_score', ascending=False, inplace=True)

print(f"Symbols after removing symbols with fewer than 100 total news and 50 recent news in 2022 & 2023: {len(summary)}")
# summary.sort_values(['avg_abs_return','count'], ascending=False, inplace=True)
summary.head(1)
# pd.reset_option('display.max_rows')

Symbols after removing symbols with fewer than 100 total news and 50 recent news in 2022 & 2023: 932


,count,avg_abs_return,max_return,min_return,recent_count,weighted_score
symbols,,,,,,
GME,502,4.622396,89.123028,-36.198214,210,51.976045


### Combine Other Information such as ETF, Industry etc from QC data

- can use sec-api to get more information such as sector, industry etc
- can use polygon to get marketcap etc

In [10]:

final_symbols_df = load_local_csv(file_path1)
final_symbols_df.head()
selected_columns = ['PrimarySymbol', 'ShortName', 'StandardName', 'Exchange', 'ListingDate', 'SectorName', 'IndustryGroupName', 'IndustryName', 'SectorETF', 'IndustryGroupETF', 'IndustryETF']
final_symbols_df = final_symbols_df[selected_columns]
final_symbols_df.set_index('PrimarySymbol', inplace=True)
final_symbols_df.head()


Loaded QCnewsSymbols.csv with 2882 rows and 33 columns


,ShortName,StandardName,Exchange,ListingDate,SectorName,IndustryGroupName,IndustryName,SectorETF,IndustryGroupETF,IndustryETF
PrimarySymbol,,,,,,,,,,
FCNCA,First Citizens BancShares,First Citizens BancShares Inc,NAS,1998-01-02,FinancialServices,Banks,BanksRegional,XLF,KBE,KRE
FMC,FMC,FMC Corp,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO
JACK,Jack In The Box,Jack In The Box Inc,NAS,1998-01-02,ConsumerCyclical,Restaurants,Restaurants,XLY,XLY,XLY
SMG,Scotts Miracle Gro,The Scotts Miracle Gro Co,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO
FLS,Flowserve,Flowserve Corp,NYS,1998-01-02,Industrials,IndustrialProducts,SpecialtyIndustrialMachinery,XLI,XLI,XLI


In [11]:
final_symbols_df = final_symbols_df[final_symbols_df.index.isin(summary.index)]
print(f"Final Symbols plus related information: {len(final_symbols_df)}")
final_symbols_df.index.name = 'ticker'
final_symbols_df.head()


Final Symbols plus related information: 928


,ShortName,StandardName,Exchange,ListingDate,SectorName,IndustryGroupName,IndustryName,SectorETF,IndustryGroupETF,IndustryETF
ticker,,,,,,,,,,
FMC,FMC,FMC Corp,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO
SMG,Scotts Miracle Gro,The Scotts Miracle Gro Co,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO
FLS,Flowserve,Flowserve Corp,NYS,1998-01-02,Industrials,IndustrialProducts,SpecialtyIndustrialMachinery,XLI,XLI,XLI
FITB,Fifth Third Bancorp,Fifth Third Bancorp,NAS,1998-01-02,FinancialServices,Banks,BanksRegional,XLF,KBE,KRE
SMTC,Semtech,Semtech Corp,NAS,1998-01-02,Technology,Semiconductors,Semiconductors,XLK,SOXX,XSD


### Polygon Marketcap, cik

In [12]:
# Not an async function since using client library - can change to async if needed - Takes ~ 40 seconds
from utils.polygonClass import Polygon
from eventtrader.keys import POLYGON_API_KEY
from tqdm import tqdm

polygon = Polygon(api_key=POLYGON_API_KEY)

polygon_ticker_details_cols = ['cik', 'composite_figi', 'currency_name', 'homepage_url', 'list_date', 'market_cap',
               'primary_exchange', 'sic_code', 'sic_description', 'share_class_shares_outstanding','share_class_figi',
                 'ticker_root', 'total_employees', 'market', 'locale']

for ticker in tqdm(final_symbols_df.index, desc="Fetching ticker details"):
    details = polygon.get_ticker_details(ticker)
    if details:
        final_symbols_df.loc[ticker, polygon_ticker_details_cols] = [getattr(details, col, None) for col in polygon_ticker_details_cols]

final_symbols_df.head()


Fetching ticker details:  68%|██████▊   | 631/928 [01:42<00:46,  6.36it/s]

Error fetching ticker details for RVNC: {"status":"NOT_FOUND","request_id":"a21a4eb203adb5ac1411a79e4a28069b","message":"Ticker not found."}


Fetching ticker details:  68%|██████▊   | 635/928 [01:42<00:40,  7.23it/s]

Error fetching ticker details for ENLC: {"status":"NOT_FOUND","request_id":"b5ab2e51b5d912373d9c9c36fde72748","message":"Ticker not found."}


Fetching ticker details:  84%|████████▍ | 780/928 [02:04<00:32,  4.56it/s]

Error fetching ticker details for SMAR: {"status":"NOT_FOUND","request_id":"23c910ec9b3e29f9f52ebb03edb9ac5f","message":"Ticker not found."}


Fetching ticker details: 100%|██████████| 928/928 [02:28<00:00,  6.26it/s]


,ShortName,StandardName,Exchange,ListingDate,SectorName,IndustryGroupName,IndustryName,SectorETF,IndustryGroupETF,IndustryETF,...,market_cap,primary_exchange,sic_code,sic_description,share_class_shares_outstanding,share_class_figi,ticker_root,total_employees,market,locale
ticker,,,,,,,,,,,,,,,,,,,,,
FMC,FMC,FMC Corp,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO,...,4.311821e+09,XNYS,2800,CHEMICALS & ALLIED PRODUCTS,124840000.0,BBG001S5R880,FMC,6600.0,stocks,us
SMG,Scotts Miracle Gro,The Scotts Miracle Gro Co,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO,...,3.866763e+09,XNYS,2870,AGRICULTURAL CHEMICALS,57470000.0,BBG001S5W5H0,SMG,5300.0,stocks,us
FLS,Flowserve,Flowserve Corp,NYS,1998-01-02,Industrials,IndustrialProducts,SpecialtyIndustrialMachinery,XLI,XLI,XLI,...,8.257012e+09,XNYS,3561,PUMPS & PUMPING EQUIPMENT,131380000.0,BBG001SB3F87,FLS,16000.0,stocks,us
FITB,Fifth Third Bancorp,Fifth Third Bancorp,NAS,1998-01-02,FinancialServices,Banks,BanksRegional,XLF,KBE,KRE,...,2.950037e+10,XNAS,6022,STATE COMMERCIAL BANKS,670540000.0,BBG001S5R6S2,FITB,18607.0,stocks,us
SMTC,Semtech,Semtech Corp,NAS,1998-01-02,Technology,Semiconductors,Semiconductors,XLK,SOXX,XSD,...,4.697612e+09,XNAS,3674,SEMICONDUCTORS & RELATED DEVICES,84530000.0,BBG001S7TWW4,SMTC,1917.0,stocks,us


### Use cik we got from Polygon and set it as index

In [13]:
final_symbols_df.set_index('cik', inplace=True, drop=False)
final_symbols_df.head()

,ShortName,StandardName,Exchange,ListingDate,SectorName,IndustryGroupName,IndustryName,SectorETF,IndustryGroupETF,IndustryETF,...,market_cap,primary_exchange,sic_code,sic_description,share_class_shares_outstanding,share_class_figi,ticker_root,total_employees,market,locale
cik,,,,,,,,,,,,,,,,,,,,,
0000037785,FMC,FMC Corp,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO,...,4.311821e+09,XNYS,2800,CHEMICALS & ALLIED PRODUCTS,124840000.0,BBG001S5R880,FMC,6600.0,stocks,us
0000825542,Scotts Miracle Gro,The Scotts Miracle Gro Co,NYS,1998-01-02,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO,...,3.866763e+09,XNYS,2870,AGRICULTURAL CHEMICALS,57470000.0,BBG001S5W5H0,SMG,5300.0,stocks,us
0000030625,Flowserve,Flowserve Corp,NYS,1998-01-02,Industrials,IndustrialProducts,SpecialtyIndustrialMachinery,XLI,XLI,XLI,...,8.257012e+09,XNYS,3561,PUMPS & PUMPING EQUIPMENT,131380000.0,BBG001SB3F87,FLS,16000.0,stocks,us
0000035527,Fifth Third Bancorp,Fifth Third Bancorp,NAS,1998-01-02,FinancialServices,Banks,BanksRegional,XLF,KBE,KRE,...,2.950037e+10,XNAS,6022,STATE COMMERCIAL BANKS,670540000.0,BBG001S5R6S2,FITB,18607.0,stocks,us
0000088941,Semtech,Semtech Corp,NAS,1998-01-02,Technology,Semiconductors,Semiconductors,XLK,SOXX,XSD,...,4.697612e+09,XNAS,3674,SEMICONDUCTORS & RELATED DEVICES,84530000.0,BBG001S7TWW4,SMTC,1917.0,stocks,us


In [14]:
final_symbols_df.index.nunique(), len(final_symbols_df.index)

(920, 928)

### Deduplciated ciks - since some ciks are duplicated?

In [15]:
# Remove duplicate ticker rows keeping first occurrence (950 unique tickers from 955 total rows)
final_symbols_df = final_symbols_df[~final_symbols_df.index.duplicated(keep='first')]
final_symbols_df.index.nunique(), len(final_symbols_df.index)

(920, 921)

### Drop columns that we got from QC and replace with columns from sec-api

In [16]:
# qc_columns = ['ShortName','ListingDate','StandardName','Exchange','SectorName', 'IndustryGroupName', 'IndustryName', 'SectorETF', 'IndustryGroupETF', 'IndustryETF']
# existing_columns = [col for col in qc_columns if col in final_symbols_df.columns]
# if existing_columns:
#     final_symbols_df.drop(columns=existing_columns, inplace=True)
# final_symbols_df.head()

### sec-api: fetching Industry, sector using 

<!-- using cik -->
https://api.sec-api.io/mapping/cik/INSERT_CIK?token=YOUR_API_KEY

<!-- using ticker -->
https://api.sec-api.io/mapping/ticker/INSERT_TICKER?token=YOUR_API_KEY


In [17]:

# Now this should work
from eventtrader.keys import POLYGON_API_KEY
from eventtrader.keys import SEC_API_KEY


ModuleNotFoundError: No module named 'sec_api'

In [18]:
import sys
import os

# Add parent directory to Python path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from sec_api.secApiClass import SECApi

# Print paths to debug if needed
print("Current directory:", os.getcwd())
print("Parent directory:", parent_dir)
print("Python path:", sys.path)

Current directory: /Users/macowne/Desktop/Faisal/EventTrader/StocksUniverse
Parent directory: /Users/macowne/Desktop/Faisal/EventTrader
Python path: ['/usr/local/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/usr/local/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/usr/local/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/macowne/Desktop/Faisal/EventTrader/venv/lib/python3.11/site-packages', '/Users/macowne/Desktop/Faisal/EventTrader']


### Takes 7 minutes

In [19]:
import asyncio
import nest_asyncio

cik_list = final_symbols_df.index.to_list()

print(f"Getting info for {len(cik_list)} ciks")
async with SECApi() as sec:
    companies_info = await sec.get_companies_info(cik_list,show_progress=True )

# Convert to DataFrame (from dict)
companies_df = pd.DataFrame.from_dict(companies_info, orient='index')
companies_df.head()

Getting info for 921 ciks


Fetching batch:   8%|▊         | 71/921 [00:27<03:35,  3.95it/s]

Error fetching SEC data for CIK nan: cannot convert float NaN to integer


Fetching batch: 100%|██████████| 921/921 [06:45<00:00,  2.27it/s]


,name,ticker,cik,cusip,exchange,isDelisted,category,sector,industry,sic,sicSector,sicIndustry,currency,location
0000037785,FMC CORP,FMC,37785,302491303,NYSE,False,Domestic Common Stock,Basic Materials,Agricultural Inputs,2800,Manufacturing,Chemicals & Allied Products,USD,Pennsylvania; U.S.A
0000825542,SCOTTS MIRACLE-GRO CO,SMG,825542,810186106,NYSE,False,Domestic Common Stock,Basic Materials,Agricultural Inputs,2870,Manufacturing,Agricultural Chemicals,USD,Ohio; U.S.A
0000030625,FLOWSERVE CORP,FLS,30625,34354P105 266849108,NYSE,False,Domestic Common Stock,Industrials,Specialty Industrial Machinery,3561,Manufacturing,Pumps & Pumping Equipment,USD,Texas; U.S.A
0000035527,FIFTH THIRD BANCORP,FITB,35527,316773100,NASDAQ,False,Domestic Common Stock Primary Class,Financial Services,Banks - Regional,6022,Finance Insurance And Real Estate,State Commercial Banks,USD,Ohio; U.S.A
0000088941,SEMTECH CORP,SMTC,88941,816850101,NASDAQ,False,Domestic Common Stock,Technology,Semiconductors,3674,Manufacturing,Semiconductors & Related Devices,USD,California; U.S.A


### Combined all columns from QC, sec-api & polygon

In [20]:
print(f"Before merge - Companies: {companies_df.shape}, Symbols: {final_symbols_df.shape}")
combined_df = pd.concat([companies_df, final_symbols_df], axis=1, join='inner')
print(f"After merge: {combined_df.shape}")
combined_df.head()

Before merge - Companies: (920, 14), Symbols: (921, 25)
After merge: (920, 39)


,name,ticker,cik,cusip,exchange,isDelisted,category,sector,industry,sic,...,market_cap,primary_exchange,sic_code,sic_description,share_class_shares_outstanding,share_class_figi,ticker_root,total_employees,market,locale
0000037785,FMC CORP,FMC,37785,302491303,NYSE,False,Domestic Common Stock,Basic Materials,Agricultural Inputs,2800,...,4.311821e+09,XNYS,2800,CHEMICALS & ALLIED PRODUCTS,124840000.0,BBG001S5R880,FMC,6600.0,stocks,us
0000825542,SCOTTS MIRACLE-GRO CO,SMG,825542,810186106,NYSE,False,Domestic Common Stock,Basic Materials,Agricultural Inputs,2870,...,3.866763e+09,XNYS,2870,AGRICULTURAL CHEMICALS,57470000.0,BBG001S5W5H0,SMG,5300.0,stocks,us
0000030625,FLOWSERVE CORP,FLS,30625,34354P105 266849108,NYSE,False,Domestic Common Stock,Industrials,Specialty Industrial Machinery,3561,...,8.257012e+09,XNYS,3561,PUMPS & PUMPING EQUIPMENT,131380000.0,BBG001SB3F87,FLS,16000.0,stocks,us
0000035527,FIFTH THIRD BANCORP,FITB,35527,316773100,NASDAQ,False,Domestic Common Stock Primary Class,Financial Services,Banks - Regional,6022,...,2.950037e+10,XNAS,6022,STATE COMMERCIAL BANKS,670540000.0,BBG001S5R6S2,FITB,18607.0,stocks,us
0000088941,SEMTECH CORP,SMTC,88941,816850101,NASDAQ,False,Domestic Common Stock,Technology,Semiconductors,3674,...,4.697612e+09,XNAS,3674,SEMICONDUCTORS & RELATED DEVICES,84530000.0,BBG001S7TWW4,SMTC,1917.0,stocks,us


### Get Related Companies list from Polygon

In [21]:
# Get list of tickers from combined_df
tickers_list = combined_df['ticker'].tolist()

# Fetch related companies with progress bar
with tqdm(total=len(tickers_list), desc="Fetching related companies", ncols=100) as pbar:
    related_companies = polygon.get_related_companies(tickers_list, pbar=pbar)

# Add the related companies as a new column to combined_df
combined_df['RelatedCompanies'] = combined_df['ticker'].map(related_companies)

# # Print some stats about the results
# print("\nSample of related companies:")
# print(combined_df[['ticker', 'RelatedCompanies']].head())

# Check for any tickers that failed to get related companies
empty_related = combined_df[combined_df['RelatedCompanies'].apply(lambda x: len(x) == 0)]
if not empty_related.empty:
    print(f"\nWarning: {len(empty_related)} tickers have no related companies")
    print("First few tickers with no related companies:", empty_related['ticker'].head().tolist())

Fetching related companies:  29%|█████████▎                      | 267/920 [00:01<00:01, 356.47it/s]

Error processing SRE: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/related-companies/SRE?apiKey=Q0npYHW_FKvSWnYhkPTDnpcEtDpkGu1s (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x119bf4190>: Failed to resolve 'api.polygon.io' ([Errno 8] nodename nor servname provided, or not known)"))
Error processing AXL: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/related-companies/AXL?apiKey=Q0npYHW_FKvSWnYhkPTDnpcEtDpkGu1s (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x119b20fd0>: Failed to resolve 'api.polygon.io' ([Errno 8] nodename nor servname provided, or not known)"))
Error processing EBAY: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/related-companies/EBAY?apiKey=Q0npYHW_FKvSWnYhkPTDnpcEtDpkGu1s (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_DECODE_ERROR] tlsv1 alert decode error (_ssl.c:1006)')

Fetching related companies:  86%|███████████████████████████▋    | 795/920 [00:02<00:00, 583.39it/s]

Error processing SXC: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/related-companies/SXC?apiKey=Q0npYHW_FKvSWnYhkPTDnpcEtDpkGu1s (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x119646d90>: Failed to resolve 'api.polygon.io' ([Errno 8] nodename nor servname provided, or not known)"))
Error processing HQY: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/related-companies/HQY?apiKey=Q0npYHW_FKvSWnYhkPTDnpcEtDpkGu1s (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x119c320d0>: Failed to resolve 'api.polygon.io' ([Errno 8] nodename nor servname provided, or not known)"))


Fetching related companies: 100%|████████████████████████████████| 920/920 [00:03<00:00, 288.26it/s]


                 RELATED COMPANIES SUMMARY                  
Total Tickers Processed: 920
Tickers with Related Companies: 737
Success Rate: 80.1%

First few tickers with no related companies: ['FMC', 'SMG', 'FUN', 'TENX', 'TFX']


### Finalize all columns

In [22]:
combined_df.columns

Index(['name', 'ticker', 'cik', 'cusip', 'exchange', 'isDelisted', 'category',
       'sector', 'industry', 'sic', 'sicSector', 'sicIndustry', 'currency',
       'location', 'ShortName', 'StandardName', 'Exchange', 'ListingDate',
       'SectorName', 'IndustryGroupName', 'IndustryName', 'SectorETF',
       'IndustryGroupETF', 'IndustryETF', 'cik', 'composite_figi',
       'currency_name', 'homepage_url', 'list_date', 'market_cap',
       'primary_exchange', 'sic_code', 'sic_description',
       'share_class_shares_outstanding', 'share_class_figi', 'ticker_root',
       'total_employees', 'market', 'locale', 'RelatedCompanies'],
      dtype='object')

In [23]:
final_list = [
    # Primary Identifiers
    'ticker', 'ticker_root', 'name', 'cusip',
    
    # Market Identifiers
    'composite_figi', 'share_class_figi', 'isDelisted', 
    
    # Exchange & Market Info
    'Exchange', 'market', 'locale', 'currency_name', 'RelatedCompanies',
    
    # Company Classification
    'SectorName', 'IndustryName', 'sic_code', 'sic_description', 
    
    # Related ETFs
    'SectorETF', 'IndustryETF',
    
    # Company Metrics
    'market_cap', 'total_employees', 'share_class_shares_outstanding',
    
    # Other
    'list_date'
]

len(final_list)

22

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

combined_df = combined_df[final_list]
combined_df.head()

,ticker,ticker_root,name,cusip,composite_figi,share_class_figi,isDelisted,Exchange,market,locale,currency_name,RelatedCompanies,SectorName,IndustryName,sic_code,sic_description,SectorETF,IndustryETF,market_cap,total_employees,share_class_shares_outstanding,list_date
0000037785,FMC,FMC,FMC CORP,302491303,BBG000BJP882,BBG001S5R880,False,NYS,stocks,us,usd,[],BasicMaterials,AgriculturalInputs,2800,CHEMICALS & ALLIED PRODUCTS,XLB,MOO,4.311821e+09,6600.0,124840000.0,1986-05-29
0000825542,SMG,SMG,SCOTTS MIRACLE-GRO CO,810186106,BBG000BT5PG5,BBG001S5W5H0,False,NYS,stocks,us,usd,[],BasicMaterials,AgriculturalInputs,2870,AGRICULTURAL CHEMICALS,XLB,MOO,3.866763e+09,5300.0,57470000.0,1992-01-31
0000030625,FLS,FLS,FLOWSERVE CORP,34354P105 266849108,BBG000FFH4P5,BBG001SB3F87,False,NYS,stocks,us,usd,"[AIT, IR, GGG, GHM, CR, ITT, EMR, PH, IR, TEX]",Industrials,SpecialtyIndustrialMachinery,3561,PUMPS & PUMPING EQUIPMENT,XLI,XLI,8.257012e+09,16000.0,131380000.0,1972-06-30
0000035527,FITB,FITB,FIFTH THIRD BANCORP,316773100,BBG000BJL3N0,BBG001S5R6S2,False,NAS,stocks,us,usd,"[CFG, KEY, MTB, USB, TFC, BKU, CMA, CFG, HBAN, RF]",FinancialServices,BanksRegional,6022,STATE COMMERCIAL BANKS,XLF,KRE,2.950037e+10,18607.0,670540000.0,1990-03-26
0000088941,SMTC,SMTC,SEMTECH CORP,816850101,BBG000DBKR53,BBG001S7TWW4,False,NAS,stocks,us,usd,"[ANET, MTSI, MKSI, DIOD, AEIS, TER]",Technology,Semiconductors,3674,SEMICONDUCTORS & RELATED DEVICES,XLK,XSD,4.697612e+09,1917.0,84530000.0,1992-03-17


### Remove delisted companies

In [25]:
print(f"Removed {len(combined_df[combined_df.isDelisted == True])} delisted companies")
combined_df = combined_df[combined_df.isDelisted == False]
combined_df.drop(columns=['isDelisted'], inplace=True)
print(f"Remaining companies: {len(combined_df)}")

Removed 5 delisted companies
Remaining companies: 915


In [26]:

print("\nMissing values for all columns (if any)")
missing = combined_df.isnull().sum()
print(pd.DataFrame({'Count': missing[missing > 0], 'Percentage': (missing[missing > 0] / len(combined_df) * 100).round(2)}).sort_values('Count', ascending=False))


Missing values for all columns (if any)
                  Count  Percentage
composite_figi       58        6.34
share_class_figi     58        6.34
total_employees      41        4.48
market_cap           38        4.15
list_date            36        3.93
sic_description      30        3.28
sic_code             29        3.17


### Filter by Marketcap > 100M
- to avoid micro-cap stocks (market manipulation potential)

In [27]:
print(f"# of Companies before filtering: {len(combined_df)}")
combined_df = combined_df[combined_df['market_cap'] > 100000000]
print(f"Marketcap > 100M: {len(combined_df)}")

# of Companies before filtering: 915
Marketcap > 100M: 875


### Remove non-compliant Industries

In [28]:

non_compliant_tickers =[ 'PM', 'MO'] # both involved in Cannabis
non_compliant_industries = ['BeveragesBrewers', 'Gambling', 'ResortsAndCasinos', 'BanksRegional', 'BanksDiversified', 'CreditServices', 'MortgageFinance']

print(f"Removing {len(combined_df[combined_df['IndustryName'].isin(non_compliant_industries)])} non-compliant industries")
print(f"Removing {len(combined_df[combined_df['ticker'].isin(non_compliant_tickers)])} non-compliant tickers")


combined_df = combined_df[~combined_df['IndustryName'].isin(non_compliant_industries)]
combined_df = combined_df[~combined_df['ticker'].isin(non_compliant_tickers)]
print(f"Remaining companies: {len(combined_df)}")


Removing 41 non-compliant industries
Removing 2 non-compliant tickers
Remaining companies: 832


### Filter by currency & locale (if not USD or US) & market ('stocks)

In [29]:
display(combined_df['currency_name'].value_counts(), combined_df['locale'].value_counts())  # Show before counts
combined_df = combined_df[(combined_df['currency_name'] == 'usd') & (combined_df['locale'] == 'us') & (combined_df['market'] == 'stocks')]
print(f"Remaining companies: {len(combined_df)}")

currency_name
usd    832
Name: count, dtype: int64

locale
us    832
Name: count, dtype: int64

Remaining companies: 832


### Filter by list date (if listed only within last 1 year since need atleast 1 year of context)

In [30]:
from datetime import datetime, timedelta
print(f"Maximum list date: {combined_df['list_date'].max()}, Minimum list date: {combined_df['list_date'].min()}")
cutoff_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
print(f"Filtering companies listed after: {cutoff_date}")
combined_df = combined_df[combined_df['list_date'] < cutoff_date]
print(f"Remaining companies: {len(combined_df)}")

Maximum list date: 2022-01-19, Minimum list date: 1892-06-23
Filtering companies listed after: 2024-02-10
Remaining companies: 832


### Rename columns

In [31]:
combined_df.columns

Index(['ticker', 'ticker_root', 'name', 'cusip', 'composite_figi',
       'share_class_figi', 'Exchange', 'market', 'locale', 'currency_name',
       'RelatedCompanies', 'SectorName', 'IndustryName', 'sic_code',
       'sic_description', 'SectorETF', 'IndustryETF', 'market_cap',
       'total_employees', 'share_class_shares_outstanding', 'list_date'],
      dtype='object')

In [32]:
column_mapping = {
    'ticker': 'symbol',
    'ticker_root': 'root',
    'name': 'company_name',
    'cusip': 'cusip',
    'composite_figi': 'figi',
    'share_class_figi': 'class_figi',
    'Exchange': 'exchange',
    'market': 'market',
    'locale': 'country',
    'currency_name': 'currency',
    'RelatedCompanies': 'related',
    'SectorName': 'sector',
    'IndustryName': 'industry',
    'sic_code': 'sic',
    'sic_description': 'sic_name',
    'SectorETF': 'sector_etf',
    'IndustryETF': 'industry_etf',
    'market_cap': 'mkt_cap',
    'total_employees': 'employees',
    'share_class_shares_outstanding': 'shares_out',
    'list_date': 'ipo_date'
}

# Rename columns
combined_df = combined_df.rename(columns=column_mapping)
combined_df.index.name = 'cik'
combined_df.head(2)

,symbol,root,company_name,cusip,figi,class_figi,exchange,market,country,currency,related,sector,industry,sic,sic_name,sector_etf,industry_etf,mkt_cap,employees,shares_out,ipo_date
cik,,,,,,,,,,,,,,,,,,,,,
0000037785,FMC,FMC,FMC CORP,302491303,BBG000BJP882,BBG001S5R880,NYS,stocks,us,usd,[],BasicMaterials,AgriculturalInputs,2800,CHEMICALS & ALLIED PRODUCTS,XLB,MOO,4.311821e+09,6600.0,124840000.0,1986-05-29
0000825542,SMG,SMG,SCOTTS MIRACLE-GRO CO,810186106,BBG000BT5PG5,BBG001S5W5H0,NYS,stocks,us,usd,[],BasicMaterials,AgriculturalInputs,2870,AGRICULTURAL CHEMICALS,XLB,MOO,3.866763e+09,5300.0,57470000.0,1992-01-31


In [33]:
### remove columns that are not needed
combined_df.drop(columns=['market', 'country', 'currency'], inplace=True)
combined_df.head(2)

,symbol,root,company_name,cusip,figi,class_figi,exchange,related,sector,industry,sic,sic_name,sector_etf,industry_etf,mkt_cap,employees,shares_out,ipo_date
cik,,,,,,,,,,,,,,,,,,
0000037785,FMC,FMC,FMC CORP,302491303,BBG000BJP882,BBG001S5R880,NYS,[],BasicMaterials,AgriculturalInputs,2800,CHEMICALS & ALLIED PRODUCTS,XLB,MOO,4.311821e+09,6600.0,124840000.0,1986-05-29
0000825542,SMG,SMG,SCOTTS MIRACLE-GRO CO,810186106,BBG000BT5PG5,BBG001S5W5H0,NYS,[],BasicMaterials,AgriculturalInputs,2870,AGRICULTURAL CHEMICALS,XLB,MOO,3.866763e+09,5300.0,57470000.0,1992-01-31


In [34]:
# with open('../News/final_symbols.txt', 'w') as f: 
#     f.write('\n'.join(summary.index.to_list()))

# print(f"Saved {len(summary)} Final symbols: {summary.index.to_list()}")

### Filtering 4 (not used to filter - just added to combined_df)

- Add predictions stats to summary

In [35]:
# # Filter combined_df to keep only symbols that exist in summary.index
# print(f"Remaining companies after symbol filtering: {len(combined_df)}")
# combined_df = combined_df[combined_df['symbol'].isin(summary.index)]
# print(f"Remaining companies after symbol filtering: {len(combined_df)}")

In [36]:
### Get LLM Predictions
predictions_df = pd.read_csv('../News/news_predictions.csv')
predictions_df.set_index('id', inplace=True)
print(f"Loaded {len(predictions_df)} existing predictions")

print(f"News rows before prediction: {len(qc_news)}")

# Filter `qc_news` to only keep rows with indices present in `predictions_df`
qc_news_filtered = qc_news.loc[qc_news.index.isin(predictions_df.index)]
qc_news_filtered['predictions'] = predictions_df.loc[qc_news_filtered.index, 'prediction']
qc_news_filtered = qc_news_filtered.dropna(subset=['predictions'])
print(f"News rows after prediction: {len(qc_news_filtered)}")


# Restrict to symbols in combined_df
qc_news_filtered = qc_news_filtered[qc_news_filtered['symbols'].isin(combined_df.symbol)]



# Calculate Win Rate - 1 for same direction, -1 for opposite direction, 0 where prediction is neutral
qc_news_filtered['win_rate'] = np.where(qc_news_filtered['predictions'] == 0, 0,
                              np.where(np.sign(qc_news_filtered['predictions']) == np.sign(qc_news_filtered['returns']), 1, -1))


# Filter out neutral predictions and calculate win rate per symbol
symbol_win_rates = (qc_news_filtered[qc_news_filtered['predictions'] != 0]
                   .groupby('symbols')
                   .agg({
                       'win_rate': lambda x: sum(x == 1) / (sum(x == 1) + sum(x == -1)),
                       'predictions': 'count'  # Total predictions for the symbol
                   })
                   .rename(columns={'predictions': 'total_predictions'}))

symbol_win_rates.sort_values('win_rate', ascending=False, inplace=True)
pd.concat([symbol_win_rates.head(3), symbol_win_rates.tail(3)])


Loaded 242696 existing predictions
News rows before prediction: 310167
News rows after prediction: 242696


/var/folders/jc/lcqcq2_942v3dvq5dz2hlv3w0000gn/T/ipykernel_26838/756012901.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qc_news_filtered['predictions'] = predictions_df.loc[qc_news_filtered.index, 'prediction']


,win_rate,total_predictions
symbols,,
DY,0.723404,94
CPA,0.723404,94
FLS,0.716981,106
MKSI,0.412281,114
PACB,0.396040,101
LESL,0.395604,91


In [37]:

# Add Win Rate
summary = summary.join(symbol_win_rates, on='symbols', how='left', lsuffix='_old', rsuffix='_new')
# Drop the old columns if they existed
summary = summary.drop(columns=[col for col in summary.columns if col.endswith('_old')], errors='ignore')
# Rename the new columns to remove suffix
summary = summary.rename(columns={col: col.replace('_new', '') for col in summary.columns if col.endswith('_new')})

summary.head(10)

,count,avg_abs_return,max_return,min_return,recent_count,weighted_score,win_rate,total_predictions
symbols,,,,,,,,
GME,502,4.622396,89.123028,-36.198214,210,51.976045,0.474537,432.0
WAL,147,4.608673,224.444161,-32.316654,94,47.840551,NaN,NaN
AMC,548,3.632101,72.520690,-24.797629,243,44.369746,0.470000,500.0
CDLX,105,7.170423,37.844256,-33.185246,63,41.896737,0.634409,93.0
ALT,236,4.560251,37.054828,-45.855549,110,41.556544,0.415888,214.0
KRYS,111,4.560902,144.928964,-18.970657,69,41.331853,0.653846,104.0
ARDX,125,4.884171,44.780812,-73.595328,70,40.989763,0.491379,116.0
CVNA,453,3.479630,49.439665,-32.231269,279,40.503475,0.532828,396.0
VKTX,118,4.453826,116.270691,-19.622325,68,39.879512,0.467890,109.0


### Predictability Filtering (not used)

In [38]:
CUTOFF = 0.70
print(f"Symbols with predicted win rate > {int(CUTOFF*100)}%: {len(summary[summary.win_rate > CUTOFF])}")
summary[summary.win_rate > CUTOFF].sort_values('win_rate', ascending=False)

Symbols with predicted win rate > 70%: 5


,count,avg_abs_return,max_return,min_return,recent_count,weighted_score,win_rate,total_predictions
symbols,,,,,,,,
DY,117,3.855730,18.928284,-16.234691,60,27.048200,0.723404,94.0
CPA,107,1.799006,9.525907,-10.526292,53,14.683977,0.723404,94.0
FLS,136,1.868360,14.005894,-7.296652,73,16.095957,0.716981,106.0
HAE,129,2.828461,15.315949,-16.709319,65,22.852875,0.714286,105.0
AMCX,145,3.204986,18.120019,-20.772626,71,26.385061,0.706349,126.0


In [39]:
len(summary)

932

In [40]:
CUTOFF = 0.60
print(f"Symbols with predicted win rate > {int(CUTOFF*100)}%: {len(summary[summary.win_rate > CUTOFF])}")
summary[summary.win_rate > CUTOFF].sort_values('win_rate', ascending=False).head(10)

Symbols with predicted win rate > 60%: 139


,count,avg_abs_return,max_return,min_return,recent_count,weighted_score,win_rate,total_predictions
symbols,,,,,,,,
DY,117,3.855730,18.928284,-16.234691,60,27.048200,0.723404,94.0
CPA,107,1.799006,9.525907,-10.526292,53,14.683977,0.723404,94.0
FLS,136,1.868360,14.005894,-7.296652,73,16.095957,0.716981,106.0
HAE,129,2.828461,15.315949,-16.709319,65,22.852875,0.714286,105.0
AMCX,145,3.204986,18.120019,-20.772626,71,26.385061,0.706349,126.0
PDCO,107,2.472657,12.180000,-14.910588,51,19.441843,0.697368,76.0
FRT,136,0.872139,4.023810,-3.343897,71,6.554198,0.696970,99.0
OGE,105,0.846251,3.856025,-5.929970,59,6.800410,0.691358,81.0
FRPT,145,2.638551,38.460623,-8.221614,90,24.875215,0.691057,123.0


### Add Summary columns to combined_df

In [41]:
# Map values from summary to combined_df
combined_df['win_rate'] = combined_df['symbol'].map(summary['win_rate'])
combined_df['total_count'] = combined_df['symbol'].map(summary['total_predictions'])
combined_df['recent_count'] = combined_df['symbol'].map(summary['recent_count'])
combined_df['avg_abs_return'] = combined_df['symbol'].map(summary['avg_abs_return'])
combined_df['max_return'] = combined_df['symbol'].map(summary['max_return'])
combined_df['min_return'] = combined_df['symbol'].map(summary['min_return'])

In [42]:

# Print summary statistics
print("\nWin Rate Statistics:")
print(combined_df['win_rate'].describe())

# Show sample of symbols and their win rates
print("\nSample of symbols and win rates:")
print(combined_df[['symbol', 'win_rate']].sort_values('win_rate', ascending=False).head())
# Print summary statistics
print("\nWin Rate Statistics:")
print(combined_df['win_rate'].describe())

# Show sample of symbols and their win rates
print("\nSample of symbols and win rates:")
print(combined_df[['symbol', 'win_rate']].sort_values('win_rate', ascending=False).head())

# Print summary statistics
print("\nWin Rate Statistics:")
print(combined_df['win_rate'].describe())

# Show sample of symbols and their win rates
print("\nSample of symbols and win rates:")
print(combined_df[['symbol', 'win_rate']].sort_values('win_rate', ascending=False).head())


Win Rate Statistics:
count    831.000000
mean       0.548881
std        0.054775
min        0.395604
25%        0.511561
50%        0.544586
75%        0.583736
max        0.723404
Name: win_rate, dtype: float64

Sample of symbols and win rates:
           symbol  win_rate
cik                        
0001345105    CPA  0.723404
0000067215     DY  0.723404
0000030625    FLS  0.716981
0000313143    HAE  0.714286
0001514991   AMCX  0.706349

Win Rate Statistics:
count    831.000000
mean       0.548881
std        0.054775
min        0.395604
25%        0.511561
50%        0.544586
75%        0.583736
max        0.723404
Name: win_rate, dtype: float64

Sample of symbols and win rates:
           symbol  win_rate
cik                        
0001345105    CPA  0.723404
0000067215     DY  0.723404
0000030625    FLS  0.716981
0000313143    HAE  0.714286
0001514991   AMCX  0.706349

Win Rate Statistics:
count    831.000000
mean       0.548881
std        0.054775
min        0.395604
25%        0

### Calculate Normalized Count (0 - 1) - score based on win rate, avg_abs_return & total_predictions


In [43]:
def calculate_normalized_score(df):
    # Normalize total_predictions to 0-1 range
    normalized_count = (df['total_predictions'] - df['total_predictions'].min()) / (df['total_predictions'].max() - df['total_predictions'].min())
    
    # Calculate final score
    df['opportunity_score'] = df['win_rate'] * df['avg_abs_return'] * normalized_count
    
    # Sort by score
    df.sort_values('opportunity_score', ascending=False, inplace=True)
    
    return df

# To use:
summary = calculate_normalized_score(summary)
print(summary[['win_rate', 'avg_abs_return', 'total_predictions', 'opportunity_score']].head())

         win_rate  avg_abs_return  total_predictions  opportunity_score
symbols                                                                
TSLA     0.515471        1.208519             5591.0           0.622957
SNAP     0.525316        2.263648              790.0           0.153951
AAPL     0.523723        0.488611             3309.0           0.150012
BA       0.527778        0.954187             1656.0           0.144277
GME      0.474537        4.622396              432.0           0.141593


In [44]:
# combined_df['score'] = combined_df['symbol'].map(summary['opportunity_score'])
combined_df.head()

,symbol,root,company_name,cusip,figi,class_figi,exchange,related,sector,industry,sic,sic_name,sector_etf,industry_etf,mkt_cap,employees,shares_out,ipo_date,win_rate,total_count,recent_count,avg_abs_return,max_return,min_return
cik,,,,,,,,,,,,,,,,,,,,,,,,
0000037785,FMC,FMC,FMC CORP,302491303,BBG000BJP882,BBG001S5R880,NYS,[],BasicMaterials,AgriculturalInputs,2800,CHEMICALS & ALLIED PRODUCTS,XLB,MOO,4.311821e+09,6600.0,124840000.0,1986-05-29,0.612565,191.0,127.0,1.267042,5.355819,-7.628036
0000825542,SMG,SMG,SCOTTS MIRACLE-GRO CO,810186106,BBG000BT5PG5,BBG001S5W5H0,NYS,[],BasicMaterials,AgriculturalInputs,2870,AGRICULTURAL CHEMICALS,XLB,MOO,3.866763e+09,5300.0,57470000.0,1992-01-31,0.575163,153.0,109.0,2.225626,9.063994,-16.911301
0000030625,FLS,FLS,FLOWSERVE CORP,34354P105 266849108,BBG000FFH4P5,BBG001SB3F87,NYS,"[AIT, IR, GGG, GHM, CR, ITT, EMR, PH, IR, TEX]",Industrials,SpecialtyIndustrialMachinery,3561,PUMPS & PUMPING EQUIPMENT,XLI,XLI,8.257012e+09,16000.0,131380000.0,1972-06-30,0.716981,106.0,73.0,1.868360,14.005894,-7.296652
0000088941,SMTC,SMTC,SEMTECH CORP,816850101,BBG000DBKR53,BBG001S7TWW4,NAS,"[ANET, MTSI, MKSI, DIOD, AEIS, TER]",Technology,Semiconductors,3674,SEMICONDUCTORS & RELATED DEVICES,XLK,XSD,4.697612e+09,1917.0,84530000.0,1992-03-17,0.509677,155.0,88.0,2.758607,18.757845,-29.120439
0000883241,SNPS,SNPS,SYNOPSYS INC,871607107,BBG000BSFRF3,BBG001S5VTR8,NAS,"[CDNS, ASUR, AZPN, ANSS, SNOW, META, WWD, PSTG, KEYS, WTS]",Technology,SoftwareInfrastructure,7372,SERVICES-PREPACKAGED SOFTWARE,XLK,IGV,8.103517e+10,20300.0,153610000.0,1992-03-03,0.521073,261.0,151.0,1.188107,9.108615,-3.558390


### Remove Symbols/Tickers not in Earningscall.biz


In [45]:
import pandas as pd

file_path1='../earningscall/earningscallSymbolList.csv'

earningscall_symbols = pd.read_csv(file_path1)
earningscall_symbols.head()

,exchange,symbol,name,sector,industry
0,NASDAQ,NDSN,Nordson Corporation - Common Stock,Industrials,Specialty Industrial Machinery
1,NASDAQ,KINS,"Kingstone Companies, Inc - Common Stock",Financial Services,Insurance - Property & Casualty
2,NYSE,ESI,Element Solutions Inc. Common Stock,Basic Materials,Specialty Chemicals
3,TSX,REAL,Real Matters Inc.,Real Estate,Real Estate Services
4,NYSE,NXRT,"NexPoint Residential Trust, Inc. Common Stock",Real Estate,REIT - Residential


In [46]:
print(f"Length before {len(combined_df)}")
combined_df = combined_df[combined_df.symbol.isin(earningscall_symbols.symbol.tolist())]
print(f"Length after {len(combined_df)}")

Length before 832
Length after 825


### Final List

In [47]:
print(f"Final symbols: {len(combined_df)}")
combined_df.to_csv('final_symbols.csv')

Final symbols: 825


In [48]:
len(qc_news[qc_news['symbols'].isin(summary.index)]), len(qc_news)

(239802, 310167)

In [49]:
# Create thresholds from 50% to 75% in 2.5% increments
thresholds = np.arange(0.40, 0.75, 0.025)
prev_count = 0

# Calculate and display counts for each threshold
for threshold in thresholds:
    count = len(combined_df[combined_df['win_rate'] > threshold])
    current_diff = prev_count - count    
    print(f"Win rate > {threshold:.1%}: {count:3d} stocks, {current_diff:3d} stocks")
    prev_count = count

Win rate > 40.0%: 822 stocks, -822 stocks
Win rate > 42.5%: 818 stocks,   4 stocks
Win rate > 45.0%: 802 stocks,  16 stocks
Win rate > 47.5%: 757 stocks,  45 stocks
Win rate > 50.0%: 662 stocks,  95 stocks
Win rate > 52.5%: 545 stocks, 117 stocks
Win rate > 55.0%: 385 stocks, 160 stocks
Win rate > 57.5%: 252 stocks, 133 stocks
Win rate > 60.0%: 139 stocks, 113 stocks
Win rate > 62.5%:  73 stocks,  66 stocks
Win rate > 65.0%:  32 stocks,  41 stocks
Win rate > 67.5%:  17 stocks,  15 stocks
Win rate > 70.0%:   5 stocks,  12 stocks
Win rate > 72.5%:   0 stocks,   5 stocks


### Final List

In [50]:
print(f"Final symbols: {len(combined_df)}")
combined_df.to_csv('final_symbols.csv')

Final symbols: 825


In [51]:
df = combined_df[combined_df['win_rate'] > 0.575]
df.sort_values('score', ascending=False, inplace=True)
print(f"Symbols with win rate > 0.5: {len(df)}")
df.head(5)


KeyError: 'score'

### final_symbols_filtered.csv: Filter with multiple conditions: Win rate > 57.5% & avg_abs_return > 2.9%

In [ ]:
win_rate_threshold = 0.575
return_threshold = 2.9
# Filter with multiple conditions and create a copy
df = combined_df[(combined_df['win_rate'] > win_rate_threshold) & (combined_df['avg_abs_return'] > return_threshold)].copy()
df = df.sort_values('score', ascending=False)
print(f"Symbols with win rate > {win_rate_threshold:.1%} & avg_abs_return > {return_threshold}: {len(df)}")

df.to_csv('final_symbols_filtered.csv')
df.head(5)

### Filter By predictability (Some ideas - not using this)

- llm predicts buy or sell or 0 (Better to do for latest news since it may have memorized old news)
- Sort by Win rate 
- Opposite direction vs S&P even better
- But first filter based on above criterias to not waste resources

In [ ]:
# Read existing predictions
predictions_df = pd.read_csv('../News/news_predictions.csv')
predictions_df.set_index('id', inplace=True)
print(f"Loaded {len(predictions_df)} existing predictions")
print("\nFirst few predictions:")
predictions_df.prediction.value_counts()


### Summary:
- combined_df[combined_df['sector'] == 'Healthcare']
- combined_df[combined_df['industry'] == 'Biotechnology']
- combined_df[combined_df['industry'] == 'DiagnosticsAndResearch']



In [58]:
from utils.fetchLocalcsv import load_local_csv 
from utils.misc import get_unique_items
import pandas as pd
import numpy as np

In [ ]:
file_path2 = '../Returns/SessionReturns_QC.csv' # Session returns calculated from Polygon
qc_news = load_local_csv(file_path2, keep_cols=['symbols', 'Unnamed: 0', 'returns', 'volume_return', 'return_SPY', 'originalTime'], na_subset=['returns'])
# Add year column
qc_news = qc_news.assign(year=lambda x: pd.to_datetime(x.originalTime, utc=True).dt.year).drop('originalTime', axis=1)
qc_news.index = qc_news['Unnamed: 0']		
qc_news.index.name = 'id'
qc_news.drop(columns=['Unnamed: 0'], inplace=True)
qc_news['Idiosyncratic_Return'] = qc_news['returns'] - qc_news['return_SPY']
qc_news['ReturnDirectionvsS&P'] = np.where((qc_news['returns'] * qc_news['return_SPY']) >= 0, 'Same', 'Opposite')
qc_news.head()

In [ ]:
qc_symbols.head()

In [ ]:
# Load original file with all columns
file_path1 = 'QCnewsSymbols.csv'
qc_symbols = load_local_csv(file_path1)

# If data exists, try mapping again
qc_symbols_unique = qc_symbols.drop_duplicates(subset='PrimarySymbol', keep='first')
qc_news['SectorName'] = qc_news['symbols'].map(qc_symbols_unique.set_index('PrimarySymbol')['SectorName'])
qc_news['IndustryName'] = qc_news['symbols'].map(qc_symbols_unique.set_index('PrimarySymbol')['IndustryName'])

qc_news.head()

In [ ]:
print(f"Symbols in qc_news: {len(qc_news)}")
qc_news = qc_news[qc_news['symbols'].isin(qc_symbols.PrimarySymbol)]
print(f"Symbols in qc_news after filtering: {len(qc_news)}")

In [ ]:
# Load predictions and merge with qc_news
predictions_df = pd.read_csv('../News/news_predictions.csv')
predictions_df.set_index('id', inplace=True)
print(f"Loaded {len(predictions_df)} existing predictions")

print(f"News rows before prediction: {len(qc_news)}")

# Filter `qc_news` to only keep rows with indices present in `predictions_df`
qc_news_filtered = qc_news.loc[qc_news.index.isin(predictions_df.index)]
qc_news_filtered['predictions'] = predictions_df.loc[qc_news_filtered.index, 'prediction']
qc_news_filtered = qc_news_filtered.dropna(subset=['predictions'])
print(f"News rows after prediction: {len(qc_news_filtered)}")

# Calculate Win Rate - 1 for same direction, -1 for opposite direction, 0 where prediction is neutral
qc_news_filtered['win_rate'] = np.where(qc_news_filtered['predictions'] == 0, 0,
                              np.where(np.sign(qc_news_filtered['predictions']) == np.sign(qc_news_filtered['returns']), 1, -1))

# Filter out neutral predictions and calculate win rate per sector/industry
symbol_win_rates = (qc_news_filtered[qc_news_filtered['predictions'] != 0]
                   .groupby(['SectorName', 'IndustryName'])
                   .agg({
                       'win_rate': lambda x: sum(x == 1) / (sum(x == 1) + sum(x == -1)),
                       'predictions': 'count'  # Total predictions for the sector/industry
                   })
                   .rename(columns={'predictions': 'total_predictions'}))

symbol_win_rates.sort_values('win_rate', ascending=False, inplace=True)
print("\nTop and bottom performing sectors/industries:")
pd.concat([symbol_win_rates.head(3), symbol_win_rates.tail(3)])

In [64]:
# Get counts per symbol
pd.set_option('display.max_rows', None)

# Group by symbols and calculate statistics including year-based counts
summary_sector = (qc_news
          .groupby('SectorName')
          .agg({
              'Idiosyncratic_Return': ['count', lambda x: abs(x).mean(), 'max', 'min'],
              'year': lambda x: sum(x.isin([2022, 2023]))  # Count of recent news
          }))


summary_industry = (qc_news
          .groupby('IndustryName')
          .agg({
              'Idiosyncratic_Return': ['count', lambda x: abs(x).mean(), 'max', 'min'],
              'year': lambda x: sum(x.isin([2022, 2023]))  # Count of recent news
          }))

summary_industry.columns = ['count', 'avg_abs_return', 'max_return', 'min_return', 'recent_count']
summary_industry.sort_values('avg_abs_return', ascending=False, inplace=True)

# Flatten column names
summary_sector.columns = ['count', 'avg_abs_return', 'max_return', 'min_return', 'recent_count']

summary_sector.sort_values('avg_abs_return', ascending=False, inplace=True)


In [65]:
# Calculate sector win rates
sector_win_rates = (qc_news_filtered[
    (qc_news_filtered['predictions'] != 0) & 
    (qc_news_filtered['SectorName'].notna())
    ]
    .groupby('SectorName')
    .agg({
        'win_rate': lambda x: sum(x == 1) / (sum(x == 1) + sum(x == -1)),
        'predictions': 'count'
    })
    .rename(columns={'predictions': 'total_predictions'}))

# Calculate industry win rates
industry_win_rates = (qc_news_filtered[
    (qc_news_filtered['predictions'] != 0) & 
    (qc_news_filtered['IndustryName'].notna())
    ]
    .groupby('IndustryName')
    .agg({
        'win_rate': lambda x: sum(x == 1) / (sum(x == 1) + sum(x == -1)),
        'predictions': 'count'
    })
    .rename(columns={'predictions': 'total_predictions'}))

# Add win rates to summary_sector and summary_industry with suffixes
summary_sector = summary_sector.join(sector_win_rates, how='left', lsuffix='_old', rsuffix='_new')
summary_industry = summary_industry.join(industry_win_rates, how='left', lsuffix='_old', rsuffix='_new')

# Drop old columns if they existed
summary_sector = summary_sector.drop(columns=[col for col in summary_sector.columns if col.endswith('_old')], errors='ignore')
summary_industry = summary_industry.drop(columns=[col for col in summary_industry.columns if col.endswith('_old')], errors='ignore')

# Rename new columns to remove suffix
summary_sector = summary_sector.rename(columns={col: col.replace('_new', '') for col in summary_sector.columns if col.endswith('_new')})
summary_industry = summary_industry.rename(columns={col: col.replace('_new', '') for col in summary_industry.columns if col.endswith('_new')})

# Get count of unique stocks per sector
stocks_per_sector = qc_symbols.groupby('SectorName')['PrimarySymbol'].nunique()

# Add this as a new column to summary_sector
summary_sector['unique_stocks'] = stocks_per_sector

# If you want to see the ratio of news per stock
summary_sector['news_per_stock'] = summary_sector['count'] / summary_sector['unique_stocks']




In [ ]:
# Calculate industry win rates
industry_win_rates = (qc_news_filtered[
    (qc_news_filtered['predictions'] != 0) & 
    (qc_news_filtered['IndustryName'].notna())
    ]
    .groupby('IndustryName')
    .agg({
        'win_rate': lambda x: sum(x == 1) / (sum(x == 1) + sum(x == -1)),
        'predictions': 'count'
    })
    .rename(columns={'predictions': 'total_predictions'}))

# Add win rates to summary_industry with suffixes
summary_industry = summary_industry.join(industry_win_rates, how='left', lsuffix='_old', rsuffix='_new')

# Drop old columns if they existed
summary_industry = summary_industry.drop(columns=[col for col in summary_industry.columns if col.endswith('_old')], errors='ignore')

# Rename new columns to remove suffix
summary_industry = summary_industry.rename(columns={col: col.replace('_new', '') for col in summary_industry.columns if col.endswith('_new')})

# Filter out rows with no predictions
summary_industry_filtered = summary_industry.dropna(subset=['win_rate'])


stocks_per_industry = qc_symbols.groupby('IndustryName')['PrimarySymbol'].nunique()

# Add this as a new column to summary_industry
summary_industry_filtered['unique_stocks'] = stocks_per_industry

# If you want to see the ratio of news per stock
summary_industry_filtered['news_per_stock'] = summary_industry_filtered['count'] / summary_industry_filtered['unique_stocks']


In [ ]:
summary_sector

In [ ]:
summary_industry_filtered